In [23]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


In [24]:
train_df=pd.read_csv(r"C:\Users\AL-AZHARY\OneDrive\Desktop\sentiement analysis\train.tsv", sep='\t', header=None)
test_df=pd.read_csv(r"C:\Users\AL-AZHARY\OneDrive\Desktop\sentiement analysis\test.tsv", sep='\t', header=None)


In [25]:
train_df.columns=['label','text']
test_df.columns=['label','text']
train_df.head()
train_df['label'].value_counts()


label
neg    28326
Name: count, dtype: int64

In [26]:
test_df.head()
test_df['label'].value_counts()


label
pos    28536
Name: count, dtype: int64

In [27]:
df=pd.concat([train_df, test_df], ignore_index=True)
df['label'].value_counts()

label
pos    28536
neg    28326
Name: count, dtype: int64

In [28]:
df.head()

,label,text
0,neg,رحمه الله رحمه واسعه واسكنه فسيح جناته 😢
1,neg,راح فين كلامك الي كنتي بتقوليه 💔 رايكو في صوتي
2,neg,كلام اهلها انهم دخلوها مستشفى الامل لمدة شهر ع...
3,neg,ليش عم تسبنا يامخرف 😡
4,neg,#السعاده_دايما_هي اشتقت ل حديثك ، و كأني لم اح...


In [29]:
df.isnull().sum()

label    0
text     0
dtype: int64

In [30]:
df_shuffled=df.sample(frac=1, random_state=42).reset_index(drop=True)
df_shuffled['label']=df_shuffled['label'].map({'pos':1,'neg':0})
df_shuffled.head(20)

,label,text
0,0,‌سر المشاهير بوزن مثالي معنا رشاقتي 🤗هي البداي...
1,0,صباح الورد😘 ميته جوع 💔 مين يعزمني عالفطور 😞 اي...
2,0,اكلم غيره 🤣
3,0,محد يعرف اسمها الحقيقي غيري اللي يبي يتفصل اعل...
4,0,كثير من البشر ليسو مخلصين بل هم مخلصين ل إحتيا...
5,0,كيف تخسر زوجتك ( تقوم قبل الفجر وأنا نايم .. ت...
6,1,"🦋 . . . . ""فيني #قليل من الفرح وأغلب تفاصيلي ز..."
7,1,ما ظنتي يحتاج مني أوصيك يا محزمي يا عزوتي ياسن...
8,1,فديتك وربي انسان عسل وذوق وكل الكلام الحلو فيه...
9,0,الله يعينهم 😶


In [31]:
x_train, x_test, y_train, y_test=train_test_split(df_shuffled['text'], df_shuffled['label'], test_size=0.2, random_state=42, stratify=df_shuffled['label'])


In [32]:
def clean_arabic(text):
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)  # تشكيل
    text = re.sub(r'[اأإآ]', 'ا', text)
    text = re.sub(r'[ى]', 'ي', text)
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [33]:
x_train_clean=x_train.apply(clean_arabic)
x_test_clean=x_test.apply(clean_arabic)

In [34]:
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(x_train_clean)

x_train_seq=pad_sequences(
    tokenizer.texts_to_sequences(x_train_clean),
    maxlen=100)
x_test_seq=pad_sequences(
    tokenizer.texts_to_sequences(x_test_clean),
    maxlen=100)

In [35]:
model_lstm = Sequential([
    Embedding(input_dim=20000, output_dim=128, input_length=100),
    LSTM(128),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_lstm.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [36]:
model_lstm.fit(
    x_train_seq, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.1
)

Epoch 1/5
640/640 [==============================] - 137s 209ms/step - loss: 0.4980 - accuracy: 0.7367 - val_loss: 0.4361 - val_accuracy: 0.7780
Epoch 2/5
640/640 [==============================] - 128s 201ms/step - loss: 0.3343 - accuracy: 0.8411 - val_loss: 0.4571 - val_accuracy: 0.7821
Epoch 3/5
640/640 [==============================] - 142s 222ms/step - loss: 0.2416 - accuracy: 0.8865 - val_loss: 0.5246 - val_accuracy: 0.7826
Epoch 4/5
640/640 [==============================] - 152s 237ms/step - loss: 0.1820 - accuracy: 0.9160 - val_loss: 0.6577 - val_accuracy: 0.7817
Epoch 5/5
640/640 [==============================] - 136s 212ms/step - loss: 0.1441 - accuracy: 0.9321 - val_loss: 0.8327 - val_accuracy: 0.7767


In [37]:
loss, accuracy = model_lstm.evaluate(x_test_seq, y_test)
print("LSTM Accuracy:", accuracy)

356/356 [==============================] - 14s 39ms/step - loss: 0.7865 - accuracy: 0.7798
LSTM Accuracy: 0.7798294425010681


In [38]:
from datasets import Dataset


train_dataset=Dataset.from_dict({
    'text':x_train.tolist(),
    'label':y_train.tolist()
})

test_dataset=Dataset.from_dict({
    'text':x_test.tolist(),
    'label':y_test.tolist()
})


In [39]:
from transformers import AutoTokenizer
import torch
tok_bert=AutoTokenizer.from_pretrained(
    r'C:\Users\AL-AZHARY\OneDrive\Desktop\sentiement analysis\bert')

def tokenize(batch):
    return tok_bert(
        batch['text'],
        padding='max_length',
        truncation=True,
        max_length=128
    )
train_dataset=train_dataset.map(tokenize, batched=True)
test_dataset=test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/45489 [00:00<?, ? examples/s]

Map:   0%|          | 0/11373 [00:00<?, ? examples/s]

In [40]:
train_dataset.set_format(
    type='torch',
    columns=['input_ids','attention_mask', 'label']
)
test_dataset.set_format(
    type='torch',
    columns=['input_ids','attention_mask', 'label']
)

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model_bert=AutoModelForSequenceClassification.from_pretrained(
    r'C:\Users\AL-AZHARY\OneDrive\Desktop\sentiement analysis\bert',
    num_labels=2,

)
training_args=TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    save_strategy="no"
)
trainer=Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()